In [13]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION = "-c search_path=dev"

MoM FFBL Weekly Rankings Replacement 

In [3]:
"""
MoM FFBL Weekly Rankings Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390"""
matchups = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(matchups_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
matchups = matchups[
    (matchups["is_playoffs"] == 0)
    & (matchups["is_consolation"] == 0)
    & (matchups["game_id"] >= 390)
]
matchups_a = matchups.copy()
matchups_b = matchups.copy()
matchups_b_cols = list(matchups_b.columns)
rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"
matchups_b.rename(columns=rename_columns, inplace=True)
matchups = pd.concat([matchups_a, matchups_b])
matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)
matchups.reset_index(drop=True, inplace=True)
matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
    ],
    axis=1,
    inplace=True,
)
matchups["win_loss"] = np.where(
    matchups["winner_team_key"] == matchups["team_a_team_key"], "W", "L"
)
matchups["2pt_system"] = matchups.groupby(["week", "game_id"])["team_a_points"].rank(
    "first", ascending=False
)
matchups["2pt_system"] = np.where(matchups["2pt_system"] <= 5, 1, 0)
matchups = matchups[
    [
        "week",
        "team_a_team_key",
        "team_a_points",
        "team_a_projected_points",
        "win_loss",
        "2pt_system",
        "game_id",
    ]
]
weeks = list(matchups["week"].unique())
regular_season = matchups[matchups["week"] == 1]
for wk in weeks[1:]:
    reg_season_week = matchups[matchups["week"] == wk]
    regular_season = regular_season.merge(
        reg_season_week,
        how="outer",
        on="team_a_team_key",
        copy=True,
        suffixes=("", f"_{wk}"),
    )
reg_season_cols = list(regular_season.columns)
twopt_cols = []
team_pts = []
team_pro_pts = []
w_l_cols = []
for col in reg_season_cols:
    if "2pt_system" in col:
        twopt_cols.append(col)
    elif "team_a_points" in col:
        team_pts.append(col)
    elif "team_a_projected_points" in col:
        team_pro_pts.append(col)
    elif "win_loss" in col:
        w_l_cols.append(col)
regular_season.insert(2, "team_a_points_total", regular_season[team_pts].sum(axis=1))
regular_season.insert(
    3, "team_a_project_points_total", regular_season[team_pro_pts].sum(axis=1)
)
regular_season.insert(
    4, "wins", regular_season[w_l_cols].apply(lambda s: (s == "W").sum(), axis=1)
)
regular_season.insert(
    5, "losses", regular_season[w_l_cols].apply(lambda s: (s == "L").sum(), axis=1)
)
twopt_cols.append("wins")
regular_season.insert(2, "2pt_total", regular_season[twopt_cols].sum(axis=1))
week_1 = regular_season["week"]
regular_season.drop("week", axis=1, inplace=True)
regular_season.insert(6, "week_1", week_1)
regular_season.insert(
    2,
    "2pt_rank",
    regular_season.groupby(["game_id"])["2pt_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    4,
    "pts_rank",
    regular_season.groupby(["game_id"])["team_a_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    6,
    "projected_pts_rank",
    regular_season.groupby(["game_id"])["team_a_project_points_total"]
    .rank(method="min", ascending=False)
    .astype(int),
)
regular_season.insert(
    0,
    "tuple",
    regular_season[["2pt_rank", "pts_rank"]].apply(tuple, axis=1),
)
regular_season.insert(
    0,
    "overall_rank",
    regular_season.groupby(["game_id"])["tuple"]
    .rank(method="min", ascending=True)
    .astype(int),
)
regular_season = regular_season.merge(
    teams, how="outer", left_on="team_a_team_key", right_on="team_key"
)
team_name = regular_season["name"]
team_mang = regular_season["nickname"]
game_id = regular_season["game_id_y"]
regular_season.drop(
    ["team_a_team_key", "name", "nickname", "tuple", "game_id_x", "game_id_y"],
    axis=1,
    inplace=True,
)
regular_season.insert(1, "manager", team_mang)
regular_season.insert(2, "team_name", team_name)
regular_season.insert(0, "game_id", game_id)
regular_season.sort_values(
    ["overall_rank", "2pt_total", "team_a_points_total"],
    ascending=[True, False, False],
    inplace=True,
)

Successfully pulled: SELECT * FROM dev.weeklyleaguematchups
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams WHERE game_id >= 390


#MoM FFBL Draft Analysis/Trends Replacement

In [20]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.leagueteams"""
draft = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION).copy_data_from_postgres(teams_query)
teams

Successfully pulled: SELECT * FROM dev.draftresults
Successfully pulled: SELECT team_key, name, nickname, game_id FROM dev.leagueteams


,team_key,name,game_id,nickname
0,406.l.103661.t.7,Harris & Harris RBs at Law 👨🏾‍⚖️,406,Greg
1,406.l.103661.t.1,Fantasy Sidelines,406,Tim
2,406.l.103661.t.2,Dak to the Future,406,Wes
3,406.l.103661.t.4,King of the Hill,406,Ryan
4,406.l.103661.t.9,Push Ups in Prison,406,Pat
5,406.l.103661.t.3,Fightin’ Blue Cock/Chubb/Wood,406,Kevin
6,406.l.103661.t.6,Wide Out University,406,Chris
7,406.l.103661.t.8,Poor Decisions 😢,406,Pete
8,406.l.103661.t.10,Team 10,406,James
9,406.l.103661.t.5,The Titanic,406,Jeremy


MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""